In [105]:
import sys
sys.path.append('/home/jovyan/work')

In [143]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from src.data_loader import cargar_oraciones_limpias, tokenize_sentences_by_char

In [144]:
# Definimos el modelo RNN
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size):
        super(RNNModel, self).__init__()
        # Capa de embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # Capa LSTM
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        # Capa de salida
        self.fc = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x):
        x = self.embedding(x)
        out, hidden = self.rnn(x)
        # Usamos la última salida del RNN
        out = self.fc(out.reshape(-1, out.size(2)))
        return out

In [145]:
# Preparar datos (convertir oraciones a índices de vocabulario)
sentences = cargar_oraciones_limpias(split="train", num_oraciones=10)
tokenized_sentences_by_char = tokenize_sentences_by_char(sentences)

In [146]:
# Crear vocabulario
vocab = list(set([char for sentence in tokenized_sentences_by_char for char in sentence]))
vocab_size = len(vocab)
char_to_idx = {char: idx for idx, char in enumerate(vocab)}

In [147]:
# Convertir las oraciones tokenizadas a índices
def encode_sentences(sentences):
    return [[char_to_idx[char] for char in sentence] for sentence in sentences]

encoded_sentences = encode_sentences(tokenized_sentences_by_char)

In [148]:
# Crear DataLoader para el entrenamiento
class CharDataset(Dataset):
    def __init__(self, sentences, seq_length):
        self.sentences = sentences
        self.seq_length = seq_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        
        # Si la oración es más corta que seq_length, rellénala con un valor de padding (por ejemplo, 0)
        padding_length = self.seq_length - len(sentence)
        
        # Aplicamos padding al final de la secuencia si es necesario
        if padding_length > 0:
            sentence = sentence + [0] * padding_length  # Añadimos padding (0) al final
        else:
            sentence = sentence[:self.seq_length]  # Si la secuencia es más larga que seq_length, truncamos
            
        # Convertir a tensor
        inputs = torch.tensor(sentence[:self.seq_length])
        targets = torch.tensor(sentence[:self.seq_length])
        
        return inputs, targets

In [149]:
# Definir la longitud de la secuencia
seq_length = 10  # Ajusta esto según lo necesites

# Filtrar oraciones que son más cortas que seq_length (si lo deseas)
filtered_sentences = [sentence for sentence in encoded_sentences if len(sentence) >= seq_length]

# Crear el dataset con las oraciones filtradas
dataset = CharDataset(filtered_sentences, seq_length)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Definir parámetros
embedding_dim = 100
hidden_dim = 128
output_size = vocab_size  # El tamaño de salida será igual al tamaño del vocabulario
epochs = 5

# Inicializamos el modelo, el criterio y el optimizador
model = RNNModel(vocab_size, embedding_dim, hidden_dim, output_size)

# Definir el criterio (CrossEntropyLoss) y el optimizador (Adam)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [150]:
# Entrenamiento del modelo
num_epochs = epochs
for epoch in range(num_epochs):
    total_loss = 0
    for inputs, targets in dataloader:
        optimizer.zero_grad()

        # Enviar inputs y targets al modelo
        outputs = model(inputs)  # El modelo toma los inputs y devuelve las predicciones

        # Verificar las formas antes de aplanar
        print(f"Outputs shape (before flattening): {outputs.shape}")  # Debería ser (batch_size, seq_length, vocab_size)
        print(f"Targets shape (before flattening): {targets.shape}")  # Debería ser (batch_size, seq_length)

        # Aplanar los outputs y targets de manera correcta
        batch_size, seq_length = targets.size()  # Esto te da las dimensiones del batch y secuencia
        
        # Aseguramos que outputs y targets tengan la misma longitud antes de la pérdida
        outputs = outputs.view(-1, vocab_size)  # Aplanar a (batch_size * seq_length, vocab_size)
        targets = targets.view(-1)  # Aplanar a (batch_size * seq_length)

        # Verificar las formas después de aplanar
        print(f"Flattened Outputs shape: {outputs.shape}")  # Debería ser (batch_size * seq_length, vocab_size)
        print(f"Flattened Targets shape: {targets.shape}")  # Debería ser (batch_size * seq_length)

        # Verificar si los tamaños coinciden
        assert outputs.shape[0] == targets.shape[0], f"Outputs size {outputs.shape[0]} does not match targets size {targets.shape[0]}"

        # Calculamos la pérdida
        loss = criterion(outputs, targets)  # Los outputs y targets deben tener el mismo tamaño
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')


Outputs shape (before flattening): torch.Size([100, 57])
Targets shape (before flattening): torch.Size([10, 10])
Flattened Outputs shape: torch.Size([100, 57])
Flattened Targets shape: torch.Size([100])
Epoch 1, Loss: 3.9436659812927246
Outputs shape (before flattening): torch.Size([100, 57])
Targets shape (before flattening): torch.Size([10, 10])
Flattened Outputs shape: torch.Size([100, 57])
Flattened Targets shape: torch.Size([100])
Epoch 2, Loss: 3.732245683670044
Outputs shape (before flattening): torch.Size([100, 57])
Targets shape (before flattening): torch.Size([10, 10])
Flattened Outputs shape: torch.Size([100, 57])
Flattened Targets shape: torch.Size([100])
Epoch 3, Loss: 3.524200201034546
Outputs shape (before flattening): torch.Size([100, 57])
Targets shape (before flattening): torch.Size([10, 10])
Flattened Outputs shape: torch.Size([100, 57])
Flattened Targets shape: torch.Size([100])
Epoch 4, Loss: 3.3202779293060303
Outputs shape (before flattening): torch.Size([100, 57

In [151]:
# Evaluar el modelo en los datos de validación
def evaluate_model(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            outputs = outputs.view(-1, vocab_size)
            targets = targets.view(-1)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(dataloader)

# Asumiendo que ya tienes un val_loader configurado de manera similar al dataloader de entrenamiento
# Puedes llamar a evaluate_model para calcular la pérdida en el conjunto de validación
val_loss = evaluate_model(model, dataloader, criterion)
print(f"Validation loss: {val_loss}")

Validation loss: 2.927612543106079


##### CARGA DE DATOS

In [96]:
# Cargamos 5 oraciones
sentences = cargar_oraciones_limpias(split="train", num_oraciones=1000)

# Aplicamos tokenización por caracteres
tokenized_sentences_by_char = tokenize_sentences_by_char(sentences)

# Clonamos para aplicar BPE 
bpe_sentences = [sentence[:] for sentence in tokenized_sentences_by_char]


##### ENTRENAMIENTO DEL MODELO N-GRAMA

In [97]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

# Función para calcular perplejidad
def calculate_perplexity(X_ngram):
    # Sumar los log de las probabilidades para cada token en X_ngram
    log_perplexity = -np.sum(np.log(X_ngram.sum(axis=1)))/X_ngram.shape[0]
    return np.exp(log_perplexity)

# Creamos un modelo n-grama simple (trabajando con caracteres)
def train_ngram_model(corpus, n):
    vectorizer = CountVectorizer(ngram_range=(n, n), tokenizer=lambda x: x.split())
    X = vectorizer.fit_transform(corpus)
    return vectorizer, X


# Preparamos los datos tokenizados por caracteres
corpus = [' '.join(sentence) for sentence in tokenized_sentences_by_char]

# Entrenamos el modelo n-grama con bigramas
vectorizer, X_ngram = train_ngram_model(corpus, 2)

# Calculamos la perplejidad para el modelo n-grama
perplexity_ngram = calculate_perplexity(X_ngram)
print(f"Perplejidad del modelo n-grama: {perplexity_ngram}")

/opt/conda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Perplejidad del modelo n-grama: 0.0067004516927731215
